In [29]:
import os
import io
import tweepy
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import glob
import os
import re

In [77]:
import nltk
#nltk.download('words')
words = set(nltk.corpus.words.words())
from nltk.corpus import stopwords
sw=set(stopwords.words('english'))

### Clean data

In [62]:
tweets = pd.read_csv("data/20220906_000000_trailrunningtweets.csv")

In [69]:
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import preprocessor as p
import numpy as np

In [78]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # remove mentions
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # remove hashtags
    temp = re.sub(r'http\S+', '', temp)  # remove links
    temp = re.sub('[()!?]', ' ', temp)  # remove punct
    temp = re.sub('\[.*?\]',' ', temp)  # remove punct
    temp = re.sub("[^a-z0-9]"," ", temp) # remove non-alphanumeric
    temp = temp.split()  # tokenize
    temp = [w for w in temp if w not in sw]  # remove stopwords
    temp = " ".join(word for word in temp)  # rejoin into tweets again
    return temp

In [79]:
df['fulltext_clean'] = df['full_text'].apply(clean_tweet)

In [82]:
df2 = df.drop('full_text', axis=1)

### Upload data

In [85]:
# use pandas to upload to GBQ
#!pip install pandas-gbq -U
import pandas_gbq

In [92]:
table_id = 'footprintsrunning.twitter_scraping.trail_running'
gbq_proj_id = 'footprintsrunning' # change this to your poroject. 
gbqclient = bigquery.Client(credentials = credentials, project=gbq_proj_id)

NameError: name 'credentials' is not defined

In [90]:
pandas_gbq.to_gbq(df2, table_id, gbq_proj_id, if_exists="replace")

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1250.17it/s]


In [91]:
table = gbqclient.get_table(table_id)  # Make an API request.

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

NameError: name 'gbqclient' is not defined

In [11]:
dataset_ref = gbqclient.dataset("twitter_scraping") # set up references to the dataset and table.
table_ref = dataset_ref.table("trail_running") # use your table name

In [12]:
job_config = bigquery.LoadJobConfig() 
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
# job_config.autodetect = True
input_file = "data/20220906_000000_trailrunningtweets.csv"

In [14]:
with open(input_file, "rb") as source_file:
    job = gbqclient.load_table_from_file(source_file, table_ref, job_config=job_config)
    
job.result()  # Waits for table load to complete.

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_ref, table_ref))

BadRequest: 400 Error while reading data, error message: Too many values in row starting at position: 128. Found 11 column(s) while expected 3.